In [ ]:
# Import necessary libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load EMNIST dataset
def load_emnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()  # Load MNIST as a placeholder
    x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
    x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32') / 255.0
    y_train = to_categorical(y_train, 10)  # Adjust for 10 classes (0-9)
    y_test = to_categorical(y_test, 10)
    return (x_train, y_train), (x_test, y_test)

# Create the model
def create_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load the data
(x_train, y_train), (x_test, y_test) = load_emnist()

# Train the model
model = create_model()
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_accuracy:.2f}')

# Function to preprocess the webcam input
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    gray = cv2.resize(gray, (28, 28))  # Resize to 28x28
    gray = gray.astype('float32') / 255.0  # Normalize
    gray = np.expand_dims(gray, axis=-1)  # Add channel dimension
    return np.expand_dims(gray, axis=0)  # Add batch dimension

# Capture input from webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame for prediction
    processed_frame = preprocess_frame(frame)

    # Predict the character
    predictions = model.predict(processed_frame)
    predicted_class = np.argmax(predictions)

    # Display the predictions on the frame
    cv2.putText(frame, f'Predicted: {predicted_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Handwritten Character Recognition', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()
